FEM-BEM Coupling
==============

The ngbem boundary element addon project initiated by Lucy Weggeler (see https://weggler.github.io/ngbem/intro.html) is now partly integrated into core NGSolve. Find a short and sweet introduction to the boundary element method there.

In this demo we simulate a plate capacitor on an unbounded domain.

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.solvers import GMRes, MinRes
from ngsolve.webgui import Draw
from ngsolve.bem import *

In [ ]:
largebox = Box ((-2,-2,-2), (2,2,2) )
eltop = Box ( (-1,-1,0.5), (1,1,1) )
elbot = Box ( (-1,-1,-1), (1,1,-0.5))

largebox.faces.name = "outer" # coupling boundary
eltop.faces.name = "el1" # Dirichlet boundary
elbot.faces.name = "el2" # Dirichlet boundary
eltop.edges.hpref = 1
elbot.edges.hpref = 1

shell = largebox-eltop-elbot # FEM domain 
shell.solids.name = "air"

mesh = shell.GenerateMesh(maxh=0.8)
mesh.RefineHP(2)
ea = { "euler_angles" : (-67, 0, 110) } 
Draw (mesh, clipping={"x":1, "y":0, "z":0, "dist" : 1.1}, **ea);

On the **exterior** domain $\Omega^c$, the solution can be expressed by the representation formula:

$$ x \in \Omega^c: \quad u(x) = - \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, \gamma_1 u (y)\, \mathrm{d}\sigma_y + \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y) , x-y\rangle }{\| x-y\|^3} } \, \gamma_0 u (y)\, \mathrm{d}\sigma_y\,, $$ 

where $\gamma_0 u = u$ and $\gamma_1 u = \frac{\partial u}{\partial n}$ are Dirichlet and Neumann traces.
These traces are related by the Calderon projector

$$  \left( \begin{array}{c} \gamma_0 u \\ \gamma_1 u \end{array}\right) =  \left( \begin{array}{cc} -V & \frac12 + K \\ \frac12 - K^\intercal & -D \end{array} \right)  \left( \begin{array}{c} \gamma_1 u \\ \gamma_0 u \end{array}\right) $$.

The $V$, $K$ are the single layer and double layer potential operators, and $D$ is the hypersingular operator.

On the FEM domain we have the variational formulation

$$
\int_{\Omega_\text{FEM}} \nabla u \nabla v \, dx - \int_\Gamma \gamma_1 u v \, ds = 0 \qquad \forall \, v \in H^1(\Omega_\text{FEM})
$$

We use Calderon's represenataion formula for the Neumann trace:

$$
\int_{\Omega_\text{FEM}} \nabla u \nabla v \, dx - \int_\Gamma \left( \left( \tfrac{1}{2} - K^\intercal\right) \,\gamma_1 u - D \, \gamma_0 u\right)  v = 0 \qquad \forall \, v \in H^1(\Omega_\text{FEM})
$$


To get a closed system, we use also the first equation of the Calderon equations. 
To see the structure of the discretized system, the dofs are split into degrees of freedom inside $\Omega$, and those on the boundary $\Gamma$. The FEM matrix $A$ is split accordingly. We see, the coupled system is symmetric, but indefinite:
  
  $$ \left( \begin{array}{ccc } A_{\Omega\Omega} & A_{\Omega\Gamma} & 0 \\ A_{\Gamma\Omega} & A_{\Gamma\Gamma } + D & -\frac12 M^\intercal + K^\intercal \\ 0 & -\frac12 M + K & -V \end{array}\right) \left( \begin{array}{c} u \\ \gamma_0 u \\ \gamma_1 u \end{array}\right) = \left( \begin{array}{c} F_{\Omega} \\ F_{\Gamma}\\ 0 \end{array}\right) \,. $$

Generate the finite element space for $H^1(\Omega)$ and set the given Dirichlet boundary conditions on the surfaces of the plates:

In [ ]:
order = 4
fesH1 = H1(mesh, order=order, dirichlet="el1|el2") 
print ("H1-ndof = ", fesH1.ndof)

The finite element space $\verb-fesH1-$ provides $H^{\frac12}(\Gamma)$ conforming element to discretize the Dirichlet trace on the coupling boundary $\Gamma$. However we still need $H^{-\frac12}(\Gamma)$ conforming elements to discretize the Neumann trace of $u$ on the coupling boundary. Here it is:

In [ ]:
fesL2 = SurfaceL2(mesh, order=order-1, dual_mapping=True, definedon=mesh.Boundaries("outer")) 
print ("L2-ndof = ", fesL2.ndof)

In [ ]:
fes = fesH1 * fesL2
u,dudn = fes.TrialFunction()
v,dvdn = fes.TestFunction()

a = BilinearForm(grad(u)*grad(v)*dx, check_unused=False).Assemble() 

gfudir = GridFunction(fes)
gfudir.components[0].Set ( mesh.BoundaryCF( { "el1" : 1, "el2" : -1 }), BND)

f = LinearForm(fes).Assemble()
res = (f.vec - a.mat * gfudir.vec).Evaluate()

Generate the the single layer potential $V$, double layer potential $K$ and hypersingular operator $D$:

In [ ]:
n = specialcf.normal(3)
with TaskManager():
    V = LaplaceSL(dudn*ds("outer"))*dvdn*ds("outer")
    K = LaplaceDL(u*ds("outer"))*dvdn*ds("outer")
    D = LaplaceSL(Cross(grad(u).Trace(),n)*ds("outer"))*Cross(grad(v).Trace(),n)*ds("outer")
    M = BilinearForm(u*dvdn*ds("outer"), check_unused=False).Assemble()

Setup the coupled system matrix and the right hand side:

In [ ]:
sym = a.mat+D.mat - (0.5*M.mat+K.mat).T - (0.5*M.mat+K.mat) - V.mat
rhs = res

bfpre = BilinearForm(grad(u)*grad(v)*dx+1e-10*u*v*dx  + dudn*dvdn*ds("outer") ).Assemble()
pre = bfpre.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

Compute the solution of the coupled system:

In [ ]:
with TaskManager():
    sol_sym = MinRes(mat=sym, rhs=rhs, pre=pre, tol=1e-8, maxsteps=200, printrates=True)

In [ ]:
gfu = GridFunction(fes)
gfu.vec[:] = sol_sym + gfudir.vec 
Draw(gfu.components[0], clipping={"x" : 1, "y":0, "z":0, "dist":0.0, "function" : True }, **ea, order=2); 

The Neumann data:

In [ ]:
Draw (gfu.components[1], **ea, order=3);